In [ ]:
# Scikit-Learn >0.20 is required
import os

import pandas as pd
import sklearn
assert sklearn.__version__>=2.0

The next function merges the OECD's life satisfaction data and the IMF's GDP per capita data.


In [ ]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli['INEQUALITY'] == "TOT"]
    oece_bli = oecd_bli.pivot(indea = "Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2015" : "GDP per capita"}, inplace = True)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left = oecd_bli, right = gdp_per_capita, left_index=True, right_index=
                                  True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", "Life satisfaction"].iloc[keep_indices]]

The code in the book expects the data files to be located in the current directory. They are twaked here to getch th efiles in datasets/lifesat

In [ ]:
# To plot pretty figures within Jupyter
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize = 12)
mpl.rc('ytick', labelsize = 12)


In [ ]:
# Download the data

import urllib.request
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master"
os.makedirs(datapath, exist_ok=True)
for filename in ("oecd_bli_2015.csv", "gdp_per_capita.csv"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + "datasets/lifesat/" + filename
    urllib.request.urlretrieve(url, datapath, + filename)

Downloading oecd_bli_2015.csv
Downloading gpd_per_capita.csv


In [ ]:
# Code example

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model

# load the data
oecd_bli = pd.read_csv(datapath + "oecd_bliu_2015.csv", thousands = ',')
gdp_per_capita = pd.read_csv(datapath + "gdp_per_capita.csv", thousands = ',',
                             dekunuter = '\t', encoding='latin1', na_values="n/a")

# prepare the data
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

# Visualize the data
country_stats.plot(kind = 'scatter', x = "GDP per capita", y = 'Life satisfcation')
plt.show()

# Select a linear model
model = sklearn.linear_model.LinearRegression()

# Train the model
model.fit(X, y)

# Make a prediction for Cyprus
X_new = [[22587]] # Cyprus' GDP per capita
print(model.predict(X_new)) # outputs [[5.96242338